### Restart and Run All Cells

In [2]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine, text
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2024
quarter = 4
today = date.today()
today_str = today.strftime("%Y-%m-%d")
today_str

'2025-02-01'

In [4]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [6]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2024 AND quarter <= 4) 
OR (year = 2024-1 AND quarter >= 4+1)
ORDER BY year DESC, quarter DESC



In [8]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.shape

(17, 5)

In [10]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2024 AND quarter <= 4-1) 
OR (year = 2024-1 AND quarter >= 4) 
ORDER BY year DESC, quarter DESC


In [14]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,3BBIF,8095,10,"2,443,671",4
1,ACE,8095,10,"750,207",4
2,ADVANC,8095,10,"32,818,977",4
3,AEONTS,8095,10,"3,228,445",4
4,AH,8095,10,"764,738",4


In [16]:
dfp.name.unique().shape

(224,)

In [18]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(226, 15)

In [20]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(394, 9)

In [22]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,BAY,2024,Q4,"29,699,751","31,155,715","-1,455,964",-4.67%
1,BBL,2024,Q4,"45,211,145","43,669,640","1,541,505",3.53%
2,FPT,2024,Q4,"1,438,028","1,636,546","-198,518",-12.13%
3,GVREIT,2024,Q4,"457,671","523,108","-65,437",-12.51%
4,KBANK,2024,Q4,"48,598,125","47,491,913","1,106,212",2.33%


In [24]:
# Create the SQL query with parameter binding
sql = text("DELETE FROM qt_profits WHERE year = :year AND quarter = :quarter")

# Execute the query with parameters
params = {'year': year, 'quarter': f'Q{quarter}'}
rp = conlt.execute(sql, params)

# Print the number of rows affected
print("Rows deleted:", rp.rowcount)

Rows deleted: 13


In [26]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.sample(5)

,name,id
276,SHANG,442
258,SABUY,721
311,SUN,667
44,BCT,729
323,TASCO,502


In [28]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
0,BAY,2024,Q4,"29,699,751","31,155,715","-1,455,964",-4.67%,49
1,BBL,2024,Q4,"45,211,145","43,669,640","1,541,505",3.53%,50
2,FPT,2024,Q4,"1,438,028","1,636,546","-198,518",-12.13%,746
3,GVREIT,2024,Q4,"457,671","523,108","-65,437",-12.51%,654
4,KBANK,2024,Q4,"48,598,125","47,491,913","1,106,212",2.33%,245


In [30]:
# Convert DataFrame to list of records
rcds = df_ins.values.tolist()

# Define column names in the same order as values
columns = ['name', 'year', 'quarter', 'latest_amt', 'previous_amt', 'inc_amt', 'inc_pct', 'ticker_id']

# SQL insert statement with named parameters
sql = text("""
    INSERT INTO qt_profits 
    (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)
    VALUES (:name, :year, :quarter, :latest_amt, :previous_amt, :inc_amt, :inc_pct, :ticker_id)
""")

try:
    # Execute inserts
    for rcd in rcds:
        # Convert list to dictionary
        params = dict(zip(columns, rcd))
        conlt.execute(sql, params)
except Exception as e:
    raise e

### End of loop

In [32]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,BAY,2024,Q4,"29,699,751","31,155,715","-1,455,964",-4.67%
1,BBL,2024,Q4,"45,211,145","43,669,640","1,541,505",3.53%
2,FPT,2024,Q4,"1,438,028","1,636,546","-198,518",-12.13%
3,GVREIT,2024,Q4,"457,671","523,108","-65,437",-12.51%
4,KBANK,2024,Q4,"48,598,125","47,491,913","1,106,212",2.33%


In [34]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,BAY,2024,Q4,"29,699,751","31,155,715","-1,455,964",-4.67%
1,BBL,2024,Q4,"45,211,145","43,669,640","1,541,505",3.53%
2,FPT,2024,Q4,"1,438,028","1,636,546","-198,518",-12.13%
3,GVREIT,2024,Q4,"457,671","523,108","-65,437",-12.51%
4,KBANK,2024,Q4,"48,598,125","47,491,913","1,106,212",2.33%


In [36]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
5,KKP,2024,Q4,"4,985,068","4,249,177","735,891",17.32%
6,KTB,2024,Q4,"43,855,657","39,491,857","4,363,800",11.05%
8,LHFG,2024,Q4,"2,046,992","1,822,134","224,858",12.34%
9,MST,2024,Q4,"462,132","378,798","83,334",22.00%
12,SCC,2024,Q4,"6,341,638","5,719,716","621,922",10.87%


In [38]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
5,KKP,2024,Q4,"4,985,068","4,249,177","735,891",17.32%
6,KTB,2024,Q4,"43,855,657","39,491,857","4,363,800",11.05%
8,LHFG,2024,Q4,"2,046,992","1,822,134","224,858",12.34%
12,SCC,2024,Q4,"6,341,638","5,719,716","621,922",10.87%


In [40]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
5,KKP,2024,Q4,"4,985,068","4,249,177","735,891",17.32%,255
8,LHFG,2024,Q4,"2,046,992","1,822,134","224,858",12.34%,268
6,KTB,2024,Q4,"43,855,657","39,491,857","4,363,800",11.05%,258
12,SCC,2024,Q4,"6,341,638","5,719,716","621,922",10.87%,427


In [42]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
5,KKP,2024,Q4,"4,985,068","4,249,177","735,891",17.32%,255
6,KTB,2024,Q4,"43,855,657","39,491,857","4,363,800",11.05%,258
8,LHFG,2024,Q4,"2,046,992","1,822,134","224,858",12.34%,268
12,SCC,2024,Q4,"6,341,638","5,719,716","621,922",10.87%,427


In [44]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
5,KKP,2024,Q4,"4,985,068","4,249,177","735,891",17.32%,255
6,KTB,2024,Q4,"43,855,657","39,491,857","4,363,800",11.05%,258
8,LHFG,2024,Q4,"2,046,992","1,822,134","224,858",12.34%,268
12,SCC,2024,Q4,"6,341,638","5,719,716","621,922",10.87%,427


In [50]:
conlt.commit()
conlt.close()